### Train the RST-LSTM

In [4]:
from tqdm import tqdm
import pandas as pd
import os
import json
import pickle

In [5]:
for fold in tqdm(range(5), desc="Writing train/test files for mask-related classification"):
    pathname = f'data/fold_{fold}'
    
    train = pickle.load(open(f'data/fold_{fold}/train.pkl', 'rb'))
    train['tree'] = train['annot'].map(lambda row: row['rst'][0])
    train['label1'] = train.masks_stance.map(str)
    train['label2'] = train.masks_argument.map(str)
    train[['text_id', 'tree', 'label1', 'label2']].to_pickle(f'data/fold_{fold}/train_mask_rst.pkl')

    test = pickle.load(open(f'data/fold_{fold}/test.pkl', 'rb'))
    test['tree'] = test['annot'].map(lambda row: row['rst'][0])
    test['label1'] = test.masks_stance.map(str)
    test['label2'] = test.masks_argument.map(str)
    test[['text_id', 'tree', 'label1', 'label2']].to_pickle(f'data/fold_{fold}/test_mask_rst.pkl')

Writing train/test files for mask-related classification: 100%|██████████| 5/5 [00:04<00:00,  1.15it/s]


In [6]:
print(test.iloc[0].tree)

id: 13
text: Иди почитай инсту наших городских пабликов где каждый 10 ый пишет что у них знакомый откуда то вернулся и плевал на карантин , у нас никто ниче не раздает , раздавать нечего в аптеках нет ничего )
proba: 1.0
relation: contrast
nuclearity: NN
left: Иди почитай инсту наших городских пабликов где каждый 10 ый пишет что у них знакомый откуда то вернулся и плевал на карантин ,
right: у нас никто ниче не раздает , раздавать нечего в аптеках нет ничего )
start: 0
end: 196


#### Config for masks 

In [8]:
%%writefile configs/rstbert_masks_0.jsonnet

local embedding_dim = 768;
local foldnum = 0;
local max_length = 512;
local lr = std.parseJson(std.extVar('lr'));
local dropout = std.parseJson(std.extVar('dropout'));
local batch_size = std.parseJson(std.extVar('batch_size'));
local treelstm_hidden_size = std.parseJson(std.extVar('treelstm_hidden_size'));
local dataset_reader_type = "models_scripts.dataset_readers.rst_tree_reader.IsaNLPRSTDatasetReader";
local model_type = "models_scripts.models.two_outputs_tree_lstm_clf.TwoOutputsTreeLSTM";
local model_name = "DeepPavlov/rubert-base-cased-conversational";

{
  vocabulary: {
    non_padded_namespaces: ["tokens", "labels1", "labels2"]
  },
  dataset_reader: {
      type: dataset_reader_type,
      num_labels1: 4,
      num_labels2: 4,
      tokenizer: {
        type: "pretrained_transformer",
        model_name: model_name,
        max_length: max_length,
      },
      token_indexers: {
          tokens: {
            type: "pretrained_transformer",
            model_name: model_name,
            max_length: max_length,
            namespace: 'tokens',
      },
    },
  },
  train_data_path: 'data/fold_' + foldnum + '/train_mask_rst.pkl',
  validation_data_path: 'data/fold_' + foldnum + '/test_mask_rst.pkl',
  model: {
    type: model_type,
    dropout: dropout,
    num_labels1: 4,
    num_labels2: 4,
    label1_weights: [0.2, 0.3, 1.0, 1.0],
    label2_weights: [0.1, 0.1, 1.0, 1.0],
    treelstm_hidden_size: treelstm_hidden_size,
    text_field_embedder: {
        token_embedders: {
            tokens: {
              type: "pretrained_transformer",
              model_name: model_name,
              max_length: max_length,
            },
        },
    },
    seq2vec_encoder: {
        type: "bert_pooler",
           pretrained_model: model_name,
           requires_grad: false,
           dropout: 0.2,
    },
    initializer: {
        regexes: [
        ["_text_field_embedder.*|_seq2vec_encoder.*|_classification_layer1.*|_classification_layer2.*",
            {
                "type": "pretrained",
                "weights_file_path": "convbert_masks/fold_" + foldnum + "/weights.th",
            }
        ],
        ],
    },
  },
  data_loader: {
    batch_sampler: {
      type: 'bucket',
      batch_size: batch_size,
    },
  },
  trainer: {
    optimizer: {
      type: "adam",
      lr: lr,
    },
    learning_rate_scheduler: {
      type: "slanted_triangular",
      cut_frac: 0.06
    },
    grad_norm: 5.0,
    grad_clipping: 5.0,
    validation_metric: '+all_mean',
    num_serialized_models_to_keep: 1,
    num_epochs: 20,
    patience: 3,
    cuda_device: 0,
  },
}

Overwriting configs/rstbert_masks_0.jsonnet


In [40]:
%%writefile configs/hparams.json

[
  {
    "type": "int",
    "attributes": {
      "name": "treelstm_hidden_size",
      "low": 50,
      "high": 100
    }
  },
  {
    "type": "int",
    "attributes": {
      "name": "batch_size",
      "low": 1,
      "high": 12
    }
  },
  {
    "type": "float",
    "attributes": {
      "name": "dropout",
      "low": 0.3,
      "high": 0.5
    }
  },
  {
    "type": "float",
    "attributes": {
      "name": "lr",
      "low": 2e-5,
      "high": 2e-3,
      "log": true
    }
  }
]

Overwriting configs/hparams.json


In [41]:
%%writefile tune_rstbert_masks.sh

export METHOD=rstbert_masks
rm -r $METHOD
mkdir $METHOD
export FOLD=0
export STUDY_NAME=rstbert_test1
allennlp tune configs/${METHOD}_${FOLD}.jsonnet configs/hparams.json --serialization-dir $METHOD/fold_${FOLD} \
    --study-name $STUDY_NAME \
    --skip-if-exists \
    --metrics best_validation_all_mean \
    --direction maximize
rm rstbert_masks/fold_0/trial_*/*.th

Overwriting tune_rstbert_masks.sh


In [11]:
import glob
import json
import os


def find_top_trials(path):
    mean_all = dict()
    for directory in glob.glob(path):
        try:
            metrics = json.load(open(os.path.join(directory, 'metrics.json'), 'r'))
            mean_all[directory] = (metrics.get('best_validation_all_mean'), 
                                   metrics.get('best_validation_f1_1_mean'),
                                   metrics.get('best_validation_f1_2_mean'))
        except:
            pass
    return {key: value for key, value in sorted(mean_all.items(), key=lambda x: x[1][0], reverse=True)}

In [48]:
ma = find_top_trials('rstbert_masks/fold_0/trial_*')
ma

In [45]:
! allennlp best-params --study-name rstbert_test1

2022-02-08 16:17:11,510 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-02-08 16:17:11,941 - INFO - allennlp.common.plugins - Plugin allennlp_optuna available
batch_size=9 dropout=0.36010392436488825 lr=0.000507610262905939 treelstm_hidden_size=76


In [58]:
%%writefile rstbert_masks.sh

export METHOD=rstbert_masks
rm -r $METHOD
mkdir $METHOD
export batch_size=4
export dropout=0.4
export lr=3e-05
export treelstm_hidden_size=94

python utils/make_k_copies.py --filename configs/${METHOD}_0.jsonnet --k 5

export FOLD=0
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_mask_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_mask_rst.pkl \
#                  --include-package models_scripts

export FOLD=1
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_mask_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_mask_rst.pkl \
#                  --include-package models_scripts

export FOLD=2
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_mask_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_mask_rst.pkl \
#                  --include-package models_scripts

export FOLD=3
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_mask_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_mask_rst.pkl \
#                  --include-package models_scripts

export FOLD=4
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_mask_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_mask_rst.pkl \
#                  --include-package models_scripts

Overwriting rstbert_masks.sh


#### Config for vaccines 

In [13]:
for fold in tqdm(range(5), desc="Writing train/test files for vaccines-related classification"):
    pathname = f'data/fold_{fold}'
    
    train = pickle.load(open(f'data/fold_{fold}/train.pkl', 'rb'))
    train['tree'] = train['annot'].map(lambda row: row['rst'][0])
    train['label1'] = train.vaccines_stance.map(str)
    train['label2'] = train.vaccines_argument.map(str)
    train[['text_id', 'tree', 'label1', 'label2']].to_pickle(f'data/fold_{fold}/train_vaccines_rst.pkl')

    test = pickle.load(open(f'data/fold_{fold}/test.pkl', 'rb'))
    test['tree'] = test['annot'].map(lambda row: row['rst'][0])
    test['label1'] = test.vaccines_stance.map(str)
    test['label2'] = test.vaccines_argument.map(str)
    test[['text_id', 'tree', 'label1', 'label2']].to_pickle(f'data/fold_{fold}/test_vaccines_rst.pkl')

Writing train/test files for vaccines-related classification: 100%|██████████| 5/5 [00:06<00:00,  1.23s/it]


In [28]:
%%writefile configs/rstbert_vaccines_0.jsonnet

local embedding_dim = 768;
local foldnum = 0;
local max_length = 512;
local lr = std.parseJson(std.extVar('lr'));
local dropout = std.parseJson(std.extVar('dropout'));
local batch_size = std.parseJson(std.extVar('batch_size'));
local treelstm_hidden_size = std.parseJson(std.extVar('treelstm_hidden_size'));
local dataset_reader_type = "models_scripts.dataset_readers.rst_tree_reader.IsaNLPRSTDatasetReader";
local model_type = "models_scripts.models.two_outputs_tree_lstm_clf.TwoOutputsTreeLSTM";
local model_name = "DeepPavlov/rubert-base-cased-conversational";

{
  vocabulary: {
    non_padded_namespaces: ["tokens", "labels1", "labels2"]
  },
  dataset_reader: {
      type: dataset_reader_type,
      num_labels1: 4,
      num_labels2: 4,
      tokenizer: {
        type: "pretrained_transformer",
        model_name: model_name,
        max_length: max_length,
      },
      token_indexers: {
          tokens: {
            type: "pretrained_transformer",
            model_name: model_name,
            max_length: max_length,
            namespace: 'tokens',
      },
    },
  },
  train_data_path: 'data/fold_' + foldnum + '/train_vaccines_rst.pkl',
  validation_data_path: 'data/fold_' + foldnum + '/test_vaccines_rst.pkl',
  model: {
    type: model_type,
    dropout: dropout,
    num_labels1: 4,
    num_labels2: 4,
    label1_weights: [0.1, 0.4, 1.0, 1.0],
    label2_weights: [0.1, 0.1, 0.6, 1.0],
    treelstm_hidden_size: treelstm_hidden_size,
    text_field_embedder: {
        token_embedders: {
            tokens: {
              type: "pretrained_transformer",
              model_name: model_name,
              max_length: max_length,
            },
        },
    },
    seq2vec_encoder: {
        type: "bert_pooler",
           pretrained_model: model_name,
           requires_grad: false,
           dropout: 0.2,
    },
    initializer: {
        regexes: [
        ["_text_field_embedder.*|_seq2vec_encoder.*|_classification_layer1.*|_classification_layer2.*",
            {
                "type": "pretrained",
                "weights_file_path": "convbert_vaccines/fold_" + foldnum + "/weights.th",
            }
        ],
        ],
    },
  },
  data_loader: {
    batch_sampler: {
      type: 'bucket',
      batch_size: batch_size,
    },
  },
  trainer: {
    optimizer: {
      type: "adam",
      lr: lr,
#       weight_decay: 0.1,
    },
    learning_rate_scheduler: {
      type: "slanted_triangular",
      cut_frac: 0.06
    },
    grad_norm: 5.0,
    grad_clipping: 5.0,
    validation_metric: '+all_mean',
    num_serialized_models_to_keep: 1,
    num_epochs: 20,
    patience: 3,
    cuda_device: 1,
  },
}

Overwriting configs/rstbert_vaccines_0.jsonnet


In [32]:
%%writefile tune_rstbert_vaccines.sh

export METHOD=rstbert_vaccines
rm -r $METHOD
mkdir $METHOD
export FOLD=0
export STUDY_NAME=rstbert_test2
# optuna delete-study $STUDY_NAME
allennlp tune configs/${METHOD}_${FOLD}.jsonnet configs/hparams.json --serialization-dir $METHOD/fold_${FOLD} \
    --study-name $STUDY_NAME \
    --skip-if-exists \
    --metrics best_validation_all_mean \
    --direction maximize
rm rstbert_vaccines/fold_0/trial_*/*.th

Overwriting tune_rstbert_vaccines.sh


In [28]:
! allennlp best-params --study-name rstbert_test2

2022-02-05 20:31:26,545 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-02-05 20:31:26,965 - INFO - allennlp.common.plugins - Plugin allennlp_optuna available
batch_size=3 dropout=0.2 lr=2.506909891571783e-05 treelstm_hidden_size=53


In [ ]:
find_top_trials('rstbert_vaccines/fold_0/trial_*')

In [56]:
%%writefile rstbert_vaccines.sh

export METHOD=rstbert_vaccines
rm -r $METHOD
mkdir $METHOD
export batch_size=3
export dropout=0.2
export lr=2.5e-05
export treelstm_hidden_size=53

python utils/make_k_copies.py --filename configs/${METHOD}_0.jsonnet --k 5

export FOLD=0
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_vaccines_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_vaccines_rst.pkl \
#                  --include-package models_scripts

export FOLD=1
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_vaccines_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_vaccines_rst.pkl \
#                  --include-package models_scripts

export FOLD=2
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_vaccines_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_vaccines_rst.pkl \
#                  --include-package models_scripts

export FOLD=3
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_vaccines_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_vaccines_rst.pkl \
#                  --include-package models_scripts

export FOLD=4
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_vaccines_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_vaccines_rst.pkl \
#                  --include-package models_scripts

Overwriting rstbert_vaccines.sh


#### Config for quarantine

In [397]:
for fold in tqdm(range(5), desc="Writing train/test files for quarantine-related classification"):
    pathname = f'data/fold_{fold}'
    
    train = pickle.load(open(f'data/fold_{fold}/train.pkl', 'rb'))
    train['tree'] = train['annot'].map(lambda row: row['rst'][0])
    train['label1'] = train.quarantine_stance.map(str)
    train['label2'] = train.quarantine_argument.map(str)
    train[['text_id', 'tree', 'label1', 'label2']].to_pickle(f'data/fold_{fold}/train_quarantine_rst.pkl')

    test = pickle.load(open(f'data/fold_{fold}/test.pkl', 'rb'))
    test['tree'] = test['annot'].map(lambda row: row['rst'][0])
    test['label1'] = test.quarantine_stance.map(str)
    test['label2'] = test.quarantine_argument.map(str)
    test[['text_id', 'tree', 'label1', 'label2']].to_pickle(f'data/fold_{fold}/test_quarantine_rst.pkl')

Writing train/test files for quarantine-related classification: 100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


In [398]:
train.quarantine_stance.map(str).value_counts()

-1    3667
1     1092
2      476
0      139
Name: quarantine_stance, dtype: int64

In [399]:
train.quarantine_argument.map(str).value_counts()

-1    3667
1     1425
2      175
0      107
Name: quarantine_argument, dtype: int64

In [36]:
%%writefile configs/rstbert_quarantine_0.jsonnet

local embedding_dim = 768;
local foldnum = 0;
local max_length = 512;
local lr = std.parseJson(std.extVar('lr'));
local dropout = std.parseJson(std.extVar('dropout'));
local batch_size = std.parseJson(std.extVar('batch_size'));
local treelstm_hidden_size = std.parseJson(std.extVar('treelstm_hidden_size'));
local dataset_reader_type = "models_scripts.dataset_readers.rst_tree_reader.IsaNLPRSTDatasetReader";
local model_type = "models_scripts.models.two_outputs_tree_lstm_clf.TwoOutputsTreeLSTM";
local model_name = "DeepPavlov/rubert-base-cased-conversational";

{
  vocabulary: {
    non_padded_namespaces: ["tokens", "labels1", "labels2"]
  },
  dataset_reader: {
      type: dataset_reader_type,
      num_labels1: 4,
      num_labels2: 4,
      tokenizer: {
        type: "pretrained_transformer",
        model_name: model_name,
        max_length: max_length,
      },
      token_indexers: {
          tokens: {
            type: "pretrained_transformer",
            model_name: model_name,
            max_length: max_length,
            namespace: 'tokens',
      },
    },
  },
  train_data_path: 'data/fold_' + foldnum + '/train_quarantine_rst.pkl',
  validation_data_path: 'data/fold_' + foldnum + '/test_quarantine_rst.pkl',
  model: {
    type: model_type,
    dropout: dropout,
    num_labels1: 4,
    num_labels2: 4,
    label1_weights: [0.1, 0.1, 0.3, 1.0],
    label2_weights: [0.1, 0.1, 0.6, 1.0],
    treelstm_hidden_size: treelstm_hidden_size,
    text_field_embedder: {
        token_embedders: {
            tokens: {
              type: "pretrained_transformer",
              model_name: model_name,
              max_length: max_length,
            },
        },
    },
    seq2vec_encoder: {
        type: "bert_pooler",
           pretrained_model: model_name,
           requires_grad: false,
           dropout: 0.2,
    },
    initializer: {
        regexes: [
        ["_text_field_embedder.*|_seq2vec_encoder.*|_classification_layer1.*|_classification_layer2.*",
            {
                "type": "pretrained",
                "weights_file_path": "convbert_quarantine/fold_" + foldnum + "/weights.th",
            }
        ],
        ],
    },
  },
  data_loader: {
    batch_sampler: {
      type: 'bucket',
      batch_size: batch_size,
    },
  },
  trainer: {
    optimizer: {
      type: "adam",
      lr: lr,
    },
    learning_rate_scheduler: {
      type: "slanted_triangular",
      cut_frac: 0.06
    },
    validation_metric: '+all_mean',
    num_serialized_models_to_keep: 1,
    num_epochs: 20,
    patience: 3,
    cuda_device: 1,
  },
}

Overwriting configs/rstbert_quarantine_0.jsonnet


In [406]:
! ls convbert_quarantine/fold_0

config.json	      metrics_epoch_4.json  out.log
meta.json	      metrics_epoch_5.json  predictions_test.json
metrics_epoch_0.json  metrics_epoch_6.json  predictions_train.json
metrics_epoch_1.json  metrics_epoch_7.json  vocabulary
metrics_epoch_2.json  metrics.json
metrics_epoch_3.json  model.tar.gz


In [1]:
%%writefile tune_rstbert_quarantine.sh

export METHOD=rstbert_quarantine
rm -r $METHOD
mkdir $METHOD
export FOLD=0
export STUDY_NAME=rstbert_test31

allennlp tune configs/${METHOD}_${FOLD}.jsonnet configs/hparams.json --serialization-dir $METHOD/fold_${FOLD} \
    --study-name $STUDY_NAME \
    --skip-if-exists \
    --metrics best_validation_all_mean \
    --direction maximize
rm rstbert_quarantine/fold_0/trial_*/*.th

Overwriting tune_rstbert_quarantine.sh


In [ ]:
find_top_trials('rstbert_quarantine/fold_0/trial_*')

In [9]:
! allennlp best-params --study-name rstbert_test31

2022-02-05 12:18:54,618 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-02-05 12:18:55,039 - INFO - allennlp.common.plugins - Plugin allennlp_optuna available
batch_size=7 dropout=0.2 lr=2.768200494663466e-05 treelstm_hidden_size=124


In [11]:
%%writefile rstbert_quarantine.sh

export METHOD=rstbert_quarantine
rm -r $METHOD
mkdir $METHOD
export batch_size=7
export dropout=0.2
export lr=2e-05
export treelstm_hidden_size=124

python utils/make_k_copies.py --filename configs/${METHOD}_0.jsonnet --k 5

export FOLD=0
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_quarantine_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_quarantine_rst.pkl \
#                  --include-package models_scripts

export FOLD=1
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_quarantine_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_quarantine_rst.pkl \
#                  --include-package models_scripts

export FOLD=2
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_quarantine_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_quarantine_rst.pkl \
#                  --include-package models_scripts

export FOLD=3
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_quarantine_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_quarantine_rst.pkl \
#                  --include-package models_scripts

export FOLD=4
allennlp train -s $METHOD/fold_${FOLD} configs/${METHOD}_${FOLD}.jsonnet
allennlp predict --use-dataset-reader --silent \
                 --output-file $METHOD/fold_${FOLD}/predictions_test.json $METHOD/fold_${FOLD}/model.tar.gz \
                 data/fold_${FOLD}/test_quarantine_rst.pkl \
                 --include-package models_scripts
                 
# allennlp predict --use-dataset-reader --silent \
#                  --output-file $METHOD/fold_${FOLD}/predictions_train.json $METHOD/fold_${FOLD}/model.tar.gz \
#                  data/fold_${FOLD}/train_quarantine_rst.pkl \
#                  --include-package models_scripts

Overwriting rstbert_quarantine.sh


### (Optional) A place to prep a model 

In [80]:
import torch

In [125]:
! cd rstbert_masks/fold_0/trial_19/ && tar -xzvf model.tar.gz

config.json
weights.th
vocabulary/
vocabulary/.lock
vocabulary/labels1.txt
vocabulary/labels2.txt
vocabulary/non_padded_namespaces.txt
meta.json


In [127]:
model = torch.load('rstbert_masks/fold_0/trial_19/weights.th', map_location='cpu')

In [ ]:
relations = ['attribution_N', 'attribution_S',
             'background_N', 'background_S',
             'cause-effect_N', 'cause-effect_S',
             'comparison_N',
             'concession_N', 'concession_S',
             'condition_N', 'condition_S',
             'contrast_N',
             'elaboration_N', 'elaboration_S',
             'evidence_N', 'evidence_S',
             'interpretation-evaluation_N', 'interpretation-evaluation_S',
             'joint_N',
             'preparation_S', 'preparation_N',
             'purpose_N', 'purpose_S',
             'restatement_N',
             'same-unit_N',
             'sequence_N',
             'solutionhood_S', 'solutionhood_N',
             'root'
             ]
weights = dict(zip(relations, model['_treelstm_encoder.W_iou.weight'][:,8:].sum(axis=0).detach().numpy()))

{key: value for key, value in sorted(weights.items(), key=lambda x: x[1], reverse=True)}

{'condition_N': 5.4683037,
 'joint_N': 5.2964687,
 'interpretation-evaluation_S': 4.8867483,
 'purpose_S': 4.861254,
 'elaboration_S': 4.0842333,
 'cause-effect_N': 3.9348679,
 'purpose_N': 3.6049266,
 'root': 3.126082,
 'restatement_N': 2.8360596,
 'evidence_N': 2.633718,
 'evidence_S': 2.5024312,
 'same-unit_N': 1.9227438,
 'contrast_N': 1.642155,
 'attribution_S': 1.4684371,
 'sequence_N': 1.4545591,
 'concession_S': 1.1923835,
 'concession_N': 1.1297396,
 'elaboration_N': 0.75989485,
 'background_S': 0.547801,
 'attribution_N': 0.39563867,
 'cause-effect_S': 0.27546582,
 'condition_S': 0.1406734,
 'preparation_N': -0.24509633,
 'preparation_S': -0.97542953,
 'solutionhood_N': -1.0200452,
 'background_N': -2.1545742,
 'comparison_N': -2.6806216,
 'solutionhood_S': -2.9214737,
 'interpretation-evaluation_N': -3.2135396}